In [6]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Define gesture classes
gesture_classes = ['0', '1', '2', '3', '4', '5', 'fist', 'I_love_you', 'ok']

# Define the directory paths
train_dir = r"C:\own_hand_gesture_dataset\train"
test_dir = r"C:\own_hand_gesture_dataset\test"

# Function to load images from a directory
def load_images_from_folder(folder):
    images = []
    labels = []
    for subfolder_name in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder_name)
        for filename in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, filename)
            img = load_img(img_path, target_size=(64, 64))
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(subfolder_name)
    return np.array(images), np.array(labels)

# Load training and testing images
X_train, y_train = load_images_from_folder(train_dir)
X_test, y_test = load_images_from_folder(test_dir)

# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Define a dictionary to map class names to integer labels
class_map = {class_name: index for index, class_name in enumerate(gesture_classes)}

# Convert class labels to integers using the class map
y_train = np.array([class_map[label] for label in y_train])
y_test = np.array([class_map[label] for label in y_test])

# Convert integer class labels to categorical
y_train = to_categorical(y_train, num_classes=len(gesture_classes))
y_test = to_categorical(y_test, num_classes=len(gesture_classes))

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(len(gesture_classes), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
model.evaluate(X_test, y_test)


Epoch 1/20
254/254 [==============================] - 13s 50ms/step - loss: 0.1746 - accuracy: 0.9465 - val_loss: 1.6689e-06 - val_accuracy: 1.0000
Epoch 2/20
254/254 [==============================] - 13s 52ms/step - loss: 7.7254e-06 - accuracy: 1.0000 - val_loss: 7.1526e-07 - val_accuracy: 1.0000
Epoch 3/20
254/254 [==============================] - 15s 58ms/step - loss: 5.4732e-06 - accuracy: 1.0000 - val_loss: 4.2385e-07 - val_accuracy: 1.0000
Epoch 4/20
254/254 [==============================] - 14s 57ms/step - loss: 3.6014e-06 - accuracy: 1.0000 - val_loss: 3.0465e-07 - val_accuracy: 1.0000
Epoch 5/20
254/254 [==============================] - 15s 60ms/step - loss: 2.4851e-06 - accuracy: 1.0000 - val_loss: 1.8544e-07 - val_accuracy: 1.0000
Epoch 6/20
254/254 [==============================] - 16s 62ms/step - loss: 2.1335e-06 - accuracy: 1.0000 - val_loss: 1.3245e-07 - val_accuracy: 1.0000
Epoch 7/20
254/254 [==============================] - 16s 61ms/step - loss: 1.7465e-06 - acc

[0.0, 1.0]

In [7]:
model.save("owndataset_hand_gesture_model.h5")

C:\Users\suriya\anaconda3\envs\env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
from sklearn.metrics import classification_report

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Predict classes for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert one-hot encoded labels to single-digit labels
y_true = np.argmax(y_test, axis=1)

# Generate classification report
report = classification_report(y_true, y_pred_classes, target_names=gesture_classes)

# Print the evaluation results
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print the classification report
print("Classification Report:")
print(report)


85/85 [==============================] - 2s 18ms/step
Test Loss: 0.0
Test Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       300
           1       1.00      1.00      1.00       300
           2       1.00      1.00      1.00       300
           3       1.00      1.00      1.00       300
           4       1.00      1.00      1.00       300
           5       1.00      1.00      1.00       300
        fist       1.00      1.00      1.00       300
  I_love_you       1.00      1.00      1.00       300
          ok       1.00      1.00      1.00       300

    accuracy                           1.00      2700
   macro avg       1.00      1.00      1.00      2700
weighted avg       1.00      1.00      1.00      2700



In [1]:

#TO DETECT HAND GESTURE IN IMAGE


import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Load the trained model
model = load_model("C:\\Users\\suriya\\anaconda3\\envs\\env\\prodigy_intern\\owndataset_hand_gesture_model.h5")

# Define gesture classes
gesture_classes = ['0', '1', '2', '3', '4', '5', 'fist', 'I_love_you', 'ok']

# Load the input image
image_path = r"C:\for testing\1.jpg" # Replace with the path to your input image
image = cv2.imread(image_path)

# Preprocess the image
resized_image = cv2.resize(image, (64, 64))
preprocessed_image = resized_image / 255.0

# Perform hand gesture recognition on the preprocessed image
preprocessed_image = np.expand_dims(preprocessed_image, axis=0)  # Add batch dimension
predictions = model.predict(preprocessed_image)

# Get the predicted class label
predicted_class_index = np.argmax(predictions)
predicted_class = gesture_classes[predicted_class_index]

# Display the predicted class label on the image
cv2.putText(image, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

# Display the image with predicted class label
cv2.imshow('Hand Gesture Recognition', image)
cv2.waitKey(0)
cv2.destroyAllWindows()





1/1 [==============================] - 1s 754ms/step


In [14]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load the trained model
model = load_model("C:\\Users\\suriya\\anaconda3\\envs\\env\\prodigy_intern\\owndataset_hand_gesture_model.h5")

# Define gesture classes
gesture_classes = ['0', '1', '2', '3', '4', '5', 'fist', 'I_love_you', 'ok']

# Path to the folder containing images
folder_path = r"C:\for testing"

# Iterate over each image in the folder
for filename in os.listdir(folder_path):
    # Load the input image
    image_path = os.path.join(folder_path, filename)
    image = cv2.imread(image_path)

    # Preprocess the image
    resized_image = cv2.resize(image, (64, 64))
    preprocessed_image = resized_image / 255.0

    # Perform hand gesture recognition on the preprocessed image
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)  # Add batch dimension
    predictions = model.predict(preprocessed_image)

    # Get the predicted class label
    predicted_class_index = np.argmax(predictions)
    predicted_class = gesture_classes[predicted_class_index]

    # Display the predicted class label on the image
    cv2.putText(image, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the image with predicted class label
    cv2.imshow('Hand Gesture Recognition', image)
    cv2.waitKey(0)

cv2.destroyAllWindows()


1/1 [==============================] - 0s 16ms/step


In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load the trained model
model = load_model("C:\\Users\\suriya\\anaconda3\\envs\\env\\prodigy_intern\\owndataset_hand_gesture_model.h5")

# Define gesture classes
gesture_classes = ['0', '1', '2', '3', '4', '5', 'fist', 'I_loveyou', 'ok']

# Open the camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    preprocessed_frame = cv2.resize(frame, (64, 64))
    preprocessed_frame = cv2.cvtColor(preprocessed_frame, cv2.COLOR_BGR2RGB)
    preprocessed_frame = img_to_array(preprocessed_frame)
    preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)
    preprocessed_frame = preprocess_input(preprocessed_frame)

    # Make predictions
    predictions = model.predict(preprocessed_frame)
    predicted_class_index = np.argmax(predictions)
    predicted_class = gesture_classes[predicted_class_index]

    # Display the predicted class on the frame
    cv2.putText(frame, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Hand Gesture Detection', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()





1/1 [==============================] - 0s 23ms/step
